Obter o PUUID

In [ ]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("RIOT_API_KEY")

if not API_KEY:
    print("Erro Crítico: A variável API_KEY está vazia APÓS carregar do .env. Verifique o arquivo .env e o nome da variável.")
    exit()

# --- DADOS DO SEU RIOT ID ---
game_name = "Lhcarva" # <--- Seu nome ANTES do #
tag_line = "2444"    # <--- Seu número DEPOIS do #

# 1. OBTER PUUID USANDO RIOT ACCOUNT V1
# Esta API usa uma região global (americas, europe, asia)
region_account = "americas" # Use 'americas' para jogadores do Brasil

url_account = f"https://{region_account}.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{game_name}/{tag_line}?api_key={API_KEY}"

print(f"DEBUG: URL da Conta Riot sendo acessada: {url_account}")

response_account = requests.get(url_account)
account_data = response_account.json()

puuid = None # Inicializa puuid como None

if response_account.status_code == 200:
    puuid = account_data.get('puuid')
    if puuid:
        print(f"\nSUCESSO! PUUID encontrado via Account V1: {puuid}")
    else:
        print("\nErro: PUUID não encontrado na resposta da Account V1 API.")
        print(account_data) # Imprime os dados para depuração
else:
    print(f"\nERRO ao buscar conta Riot: Código {response_account.status_code}")
    print(f"Mensagem de erro: {account_data.get('status', {}).get('message', 'Nenhuma mensagem de erro específica.')}")
    print("Verifique seu game_name e tag_line, e se a chave API é válida.")

# 2. SE O PUUID FOI OBTIDO, AGORA PODEMOS BUSCAR DADOS DO INVOCADOR USANDO SUMMONER V4 /BY-PUUID
# Esta API volta a usar a região específica do servidor (br1, euw1, etc.)
if puuid:
    region_summoner = "br1" # <--- Sua região do servidor de LoL

    url_summoner_by_puuid = f"https://{region_summoner}.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/{puuid}?api_key={API_KEY}"

    print(f"DEBUG: URL do Invocador por PUUID sendo acessada: {url_summoner_by_puuid}")

    response_summoner_by_puuid = requests.get(url_summoner_by_puuid)
    summoner_data = response_summoner_by_puuid.json()

    if response_summoner_by_puuid.status_code == 200:
        print("\nSUCESSO! Dados do invocador obtidos:")
        print(f"Nome de Invocador: {summoner_data.get('name')}")
        print(f"Level: {summoner_data.get('summonerLevel')}")
        # Você pode imprimir mais dados aqui
    else:
        print(f"\nERRO ao buscar invocador por PUUID: Código {response_summoner_by_puuid.status_code}")
        print(f"Mensagem de erro: {summoner_data.get('status', {}).get('message', 'Nenhuma mensagem de erro específica.')}")
else:
    print("\nPUUID não foi obtido, não é possível buscar dados do invocador.")

# ... Agora você pode usar o 'puuid' para buscar Match V5 como planejado ...
# Exemplo para Match V5:
# if puuid:
#     region_matches = "americas" # Match V5 também usa regiões globais
#     url_match_ids = f"https://{region_matches}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=20&api_key={API_KEY}"
#     print(f"DEBUG: URL de Match IDs: {url_match_ids}")
#     # ... faça a requisição para obter os IDs das partidas ...

Puxar 100 partidas

In [ ]:
import requests
import os
from dotenv import load_dotenv
import time
import json
import csv
# Não precisamos importar datetime explicitamente se apenas manipulamos timestamps brutos

# --- Configurações dos Arquivos CSV ---
MATCHES_CSV_FILE = 'matches_kuri.csv'
PARTICIPANTS_CSV_FILE = 'participants_kuri.csv'

# --- FUNÇÃO PARA ESCREVER CABEÇALHOS NOS ARQUIVOS CSV ---
def write_csv_headers():
    """
    Escreve os cabeçalhos nas tabelas CSV se os arquivos não existirem ou estiverem vazios.
    Esta função deve ser chamada apenas uma vez por execução do script.
    """
    # Cabeçalhos para o arquivo de partidas (matches.csv)
    # 'game_creation' é o timestamp de início. 'endTime' é calculado.
    match_headers = [
        'match_id', 'game_creation', 'game_duration', 'game_mode',
        'game_version', 'queue_id', 'platform_id',
        'endTime', # 'endTime' é o timestamp de fim da partida
        'mapId', 'gameType'
    ]

    # Cabeçalhos para o arquivo de participantes (participants.csv)
    participant_headers = [
        'participant_entry_id', 'match_id', 'puuid', 'summoner_name',
        'champion_name', 'team_id', 'win', 'kills', 'deaths', 'assists',
        'gold_earned', 'total_damage_dealt_to_champions', 'vision_score',
        'wardsPlaced', 'wardsKilled', 'cs', 'individualPosition',
        'goldSpent', 'totalDamageDealt', 'magicDamageDealtToChampions',
        'physicalDamageDealtToChampions', 'trueDamageDealtToChampions',
        'totalDamageTaken', 'damageDealtToObjectives', 'damageDealtToTurrets',
        'detectorWardsPlaced', 'visionWardsBoughtInGame', 'firstBloodKill',
        'firstBloodAssist', 'firstTowerKill', 'firstTowerAssist',
        'doubleKills', 'tripleKills', 'quadraKills', 'pentaKills',
        'timeCCingOthers', 'objectivesStolen', 'baronKills', 'dragonKills',
        'turretKills', 'lane', 'role',
        # Colunas do objeto 'challenges'
        'killParticipation', 'damagePerMinute', 'goldPerMinute',
        'visionScorePerMinute', 'soloKills'
    ]

    # Abre o arquivo de partidas em modo 'a' (append) para adicionar.
    # 'newline=""' é importante para evitar linhas em branco extras no CSV.
    # 'encoding="utf-8"' para lidar com caracteres especiais.
    with open(MATCHES_CSV_FILE, 'a', newline='', encoding='utf-8') as f_matches:
        writer_matches = csv.writer(f_matches)
        # Checa se o arquivo está vazio (posição do cursor é 0) para escrever o cabeçalho.
        if f_matches.tell() == 0:
            writer_matches.writerow(match_headers)

    # Faz o mesmo para o arquivo de participantes.
    with open(PARTICIPANTS_CSV_FILE, 'a', newline='', encoding='utf-8') as f_participants:
        writer_participants = csv.writer(f_participants)
        if f_participants.tell() == 0:
            writer_participants.writerow(participant_headers)

    print("Cabeçalhos dos arquivos CSV verificados/escritos.")


# --- FUNÇÃO PARA SALVAR DADOS DE PARTIDA E PARTICIPANTES EM CSV ---
def save_match_data_to_csv(match_data, participant_id_counter):
    """
    Extrai os dados relevantes de um objeto JSON de partida da Riot API
    e os salva em arquivos CSV separados para partidas e participantes.
    Retorna True se a operação foi bem-sucedida, False caso contrário,
    e o contador de ID de participante atualizado.
    """
    info = match_data.get('info', {})
    metadata = match_data.get('metadata', {})
    match_id = metadata.get('matchId')

    # Se não houver matchId (problema nos dados da API), ignora esta partida.
    if not match_id:
        print("Aviso: matchId não encontrado nos dados da partida. Ignorando.")
        return False, participant_id_counter

    try:
        # Extrair dados de tempo principais
        game_creation_ms = info.get('gameCreation') # Timestamp de início em milissegundos
        game_duration_s = info.get('gameDuration')   # Duração em segundos

        # Calcular endTime em milissegundos
        end_time_ms = None
        if game_creation_ms is not None and game_duration_s is not None:
            end_time_ms = game_creation_ms + (game_duration_s * 1000) # Converter duração para ms

        # --- Escreve no matches.csv ---
        with open(MATCHES_CSV_FILE, 'a', newline='', encoding='utf-8') as f_matches:
            writer_matches = csv.writer(f_matches)
            match_row = [
                match_id,
                game_creation_ms,
                game_duration_s,
                info.get('gameMode'),
                info.get('gameVersion'),
                info.get('queueId'),
                info.get('platformId'),
                end_time_ms,
                info.get('mapId'),
                info.get('gameType')
            ]
            writer_matches.writerow(match_row)

        # --- Escreve no participants.csv ---
        with open(PARTICIPANTS_CSV_FILE, 'a', newline='', encoding='utf-8') as f_participants:
            writer_participants = csv.writer(f_participants)
            for participant in info.get('participants', []):
                # Cada participante ganha um ID sequencial único nesta execução do script
                participant_id_counter += 1

                # Converte o booleano 'win' (True/False) para 1 (vitória) ou 0 (derrota)
                win_status = 1 if participant.get('win', False) else 0
                
                # Calcula o CS total somando minions e monstros
                cs = participant.get('totalMinionsKilled', 0) + participant.get('neutralMinionsKilled', 0)
                
                # Acessa o objeto 'challenges', usando um dicionário vazio como padrão se não existir
                challenges = participant.get('challenges', {})

                participant_row = [
                    participant_id_counter,
                    match_id, # Chave estrangeira para a partida
                    participant.get('puuid'),
                    participant.get('summonerName'),
                    participant.get('championName'),
                    participant.get('teamId'),
                    win_status,
                    participant.get('kills'),
                    participant.get('deaths'),
                    participant.get('assists'),
                    participant.get('goldEarned'),
                    participant.get('totalDamageDealtToChampions'),
                    participant.get('visionScore'),
                    participant.get('wardsPlaced'),
                    participant.get('wardsKilled'),
                    cs,
                    participant.get('individualPosition'),
                    # Novas colunas adicionadas para granularidade de análise
                    participant.get('goldSpent'),
                    participant.get('totalDamageDealt'),
                    participant.get('magicDamageDealtToChampions'),
                    participant.get('physicalDamageDealtToChampions'),
                    participant.get('trueDamageDealtToChampions'),
                    participant.get('totalDamageTaken'),
                    participant.get('damageDealtToObjectives'),
                    participant.get('damageDealtToTurrets'),
                    participant.get('detectorWardsPlaced'),
                    participant.get('visionWardsBoughtInGame'),
                    1 if participant.get('firstBloodKill') else 0, # Converte boolean para 0/1
                    1 if participant.get('firstBloodAssist') else 0,
                    1 if participant.get('firstTowerKill') else 0,
                    1 if participant.get('firstTowerAssist') else 0,
                    participant.get('doubleKills'),
                    participant.get('tripleKills'),
                    participant.get('quadraKills'),
                    participant.get('pentaKills'),
                    participant.get('timeCCingOthers'),
                    participant.get('objectivesStolen'),
                    participant.get('baronKills'),
                    participant.get('dragonKills'),
                    participant.get('turretKills'),
                    participant.get('lane'),
                    participant.get('role'),
                    # Valores extraídos do objeto 'challenges'
                    challenges.get('killParticipation'),
                    challenges.get('damagePerMinute'),
                    challenges.get('goldPerMinute'),
                    challenges.get('visionScorePerMinute'),
                    challenges.get('soloKills')
                ]
                writer_participants.writerow(participant_row)
        return True, participant_id_counter
    except Exception as e:
        print(f"Erro ao salvar dados da partida {match_id} no CSV: {e}")
        return False, participant_id_counter


# --- CÓDIGO PRINCIPAL PARA EXECUTAR O PROCESSO DE COLETA ---
def main():
    load_dotenv() # Carrega as variáveis de ambiente do arquivo .env
    API_KEY = os.getenv("RIOT_API_KEY")

    if not API_KEY:
        print("Erro Crítico: A variável API_KEY está vazia APÓS carregar do .env. Verifique o arquivo .env.")
        return

    # Contador para IDs sequenciais de participantes (reinicia a cada execução do script)
    participant_id_counter = 0

    # Garante que os arquivos CSV e seus cabeçalhos existam ou sejam criados.
    # ATENÇÃO: Se cabeçalhos mudaram, você precisa DELETAR os CSVs antigos antes de rodar!
    write_csv_headers()

    # --- INFORMAÇÕES DO SEU JOGADOR ---
    # Substitua pelo seu Game Name e Tag Line EXATOS.
    game_name = "PAIN Kuri"
    tag_line = "LTAS"
    region_account = "americas" # Região global para a API de Account (BR usa americas)

    # --- OBTER O PUUID DO JOGADOR ---
    # O PUUID é um identificador único e global do jogador na Riot.
    url_account = f"https://{region_account}.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{game_name}/{tag_line}?api_key={API_KEY}"
    print(f"\nBuscando PUUID para {game_name}#{tag_line}: {url_account}")
    response_account = requests.get(url_account)
    account_data = response_account.json()

    puuid = None
    if response_account.status_code == 200:
        puuid = account_data.get('puuid')
        print(f"PUUID obtido: {puuid}")
    else:
        print(f"ERRO ao buscar conta Riot: Código {response_account.status_code} - {account_data.get('status', {}).get('message', 'Nenhuma mensagem de erro específica.')}")
        return # Encerrar se não conseguir o PUUID

    if not puuid:
        print("PUUID não obtido. Não é possível buscar partidas.")
        return # Encerrar se o PUUID for nulo

    # --- CONFIGURAÇÕES DE BUSCA DE PARTIDAS ---
    region_matches = "americas" # Região global para a API de Match (BR usa americas)
    # Pegamos 50 de cada tipo para garantir até 100 partidas únicas no total (50 Solo/Duo + 50 Flex)
    count_per_queue = 100
    
    # IDs das filas ranqueadas que queremos buscar
    RANKED_SOLODUO_QUEUE_ID = 420 # Ranqueada Solo/Duo
    #RANKED_FLEX_QUEUE_ID = 440    # Ranqueada Flex 5x5

    # --- LISTA PARA ARMAZENAR OS IDs DAS PARTIDAS ÚNICAS ---
    # Usamos um conjunto (set) para garantir que não haja IDs duplicados ao combinar Solo/Duo e Flex
    all_match_ids = set() 

    # 1. Buscar IDs de partidas Solo/Duo
    url_solo_duo_ids = f"https://{region_matches}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count={count_per_queue}&queue={RANKED_SOLODUO_QUEUE_ID}&api_key={API_KEY}"
    print(f"\nBuscando IDs de {count_per_queue} partidas Solo/Duo: {url_solo_duo_ids}")
    response_solo_duo_ids = requests.get(url_solo_duo_ids)
    solo_duo_ids = response_solo_duo_ids.json()

    if response_solo_duo_ids.status_code == 200:
        print(f"IDs de Solo/Duo encontradas: {len(solo_duo_ids)}")
        all_match_ids.update(solo_duo_ids) # Adiciona ao conjunto de IDs únicos
    else:
        print(f"Erro ao buscar IDs de Solo/Duo: {response_solo_duo_ids.status_code} - {solo_duo_ids.get('status', {}).get('message', '')}")
    
    # Pequena pausa antes da próxima requisição de IDs para evitar bater o limite rápido demais
    time.sleep(1.2) 

    '''# 2. Buscar IDs de partidas Flex
    url_flex_ids = f"https://{region_matches}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count={count_per_queue}&queue={RANKED_FLEX_QUEUE_ID}&api_key={API_KEY}"
    print(f"\nBuscando IDs de {count_per_queue} partidas Flex: {url_flex_ids}")
    response_flex_ids = requests.get(url_flex_ids)
    flex_ids = response_flex_ids.json()

    if response_flex_ids.status_code == 200:
        print(f"IDs de Flex encontradas: {len(flex_ids)}")
        all_match_ids.update(flex_ids) # Adiciona ao conjunto de IDs únicos
    else:
        print(f"Erro ao buscar IDs de Flex: {response_flex_ids.status_code} - {flex_ids.get('status', {}).get('message', '')}")'''

    # Converte o conjunto de IDs únicos para uma lista para iterar
    final_match_ids_to_process = list(all_match_ids)
    print(f"\nTotal de IDs de partidas únicas para processar (Solo/Duo + Flex): {len(final_match_ids_to_process)}")

    processed_matches_count = 0
    # Atraso fixo por requisição de detalhes de partida (100 requisições / 120 segundos = 1.2 segundos/requisição)
    DELAY_BETWEEN_MATCH_DETAILS_REQUESTS = 1.2 

    # --- LOOP PARA OBTER DETALHES DE CADA PARTIDA E SALVAR EM CSV ---
    for i, match_id in enumerate(final_match_ids_to_process):
        print(f"Processando partida {i+1}/{len(final_match_ids_to_process)}: {match_id}")

        # Pausa antes de CADA requisição de detalhes de partida para respeitar os limites
        print(f"Aguardando {DELAY_BETWEEN_MATCH_DETAILS_REQUESTS} segundos antes da próxima requisição de detalhes...")
        time.sleep(DELAY_BETWEEN_MATCH_DETAILS_REQUESTS) 

        url_match_data = f"https://{region_matches}.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={API_KEY}"
        response_match_data = requests.get(url_match_data)
        match_data = response_match_data.json()

        if response_match_data.status_code == 200:
            success, participant_id_counter = save_match_data_to_csv(match_data, participant_id_counter)
            if success:
                processed_matches_count += 1
        else:
            print(f"Erro ao buscar dados da partida {match_id}: {response_match_data.status_code} - {match_data.get('status', {}).get('message', '')}")
            if response_match_data.status_code == 429: # Limite de taxa excedido para detalhes
                print(f"Limite de taxa da API excedido para detalhes da partida. Pausando por 120 segundos. A execução atual será interrompida. Por favor, rode o script novamente após o tempo de espera para continuar.")
                time.sleep(120) # Pausa para que o limite da API se reinicie
                break # Interrompe o loop de processamento de detalhes para evitar mais erros

    print(f"\n--- Processamento concluído. {processed_matches_count} partidas salvas nos arquivos CSV. ---")

# Garante que a função main() seja chamada apenas quando o script é executado diretamente
if __name__ == "__main__":
    main()


Unificar os CSVs

In [ ]:
import csv
import os

def combine_csv_files(input_file1, input_file2, output_file):
    """
    Combina o conteúdo de dois arquivos CSV em um único arquivo CSV.
    Assume que ambos os arquivos têm o mesmo cabeçalho.
    O cabeçalho é copiado apenas do primeiro arquivo.
    """
    all_rows = []
    
    # 1. Lê o primeiro arquivo (inclui o cabeçalho)
    try:
        with open(input_file1, 'r', newline='', encoding='utf-8') as f1:
            reader1 = csv.reader(f1)
            header = next(reader1)  # Lê e armazena o cabeçalho
            all_rows.append(header) # Adiciona o cabeçalho à lista de todas as linhas
            for row in reader1:
                all_rows.append(row)
        print(f"Lido {len(all_rows) - 1} linhas de dados de '{input_file1}'.")
    except FileNotFoundError:
        print(f"Erro: Arquivo '{input_file1}' não encontrado.")
        return False
    except Exception as e:
        print(f"Erro ao ler '{input_file1}': {e}")
        return False

    # 2. Lê o segundo arquivo (ignora o cabeçalho)
    try:
        with open(input_file2, 'r', newline='', encoding='utf-8') as f2:
            reader2 = csv.reader(f2)
            next(reader2)  # Pula o cabeçalho do segundo arquivo
            rows_added = 0
            for row in reader2:
                all_rows.append(row)
                rows_added += 1
        print(f"Lido {rows_added} linhas de dados de '{input_file2}'.")
    except FileNotFoundError:
        print(f"Erro: Arquivo '{input_file2}' não encontrado.")
        return False
    except Exception as e:
        print(f"Erro ao ler '{input_file2}': {e}")
        return False

    # 3. Escreve todas as linhas no novo arquivo de saída
    try:
        with open(output_file, 'w', newline='', encoding='utf-8') as outfile:
            writer = csv.writer(outfile)
            writer.writerows(all_rows)
        print(f"Arquivos combinados com sucesso em '{output_file}'. Total de {len(all_rows) - 1} linhas de dados.")
        return True
    except Exception as e:
        print(f"Erro ao escrever em '{output_file}': {e}")
        return False

# --- EXEMPLOS DE USO ---
if __name__ == "__main__":
    # Assumindo que seus arquivos estão na mesma pasta do script
    
    # Exemplo 1: Combinar arquivos de partidas (matches.csv)
    my_matches_file = 'matches.csv'      # Nome do CSV com suas partidas
    titan_matches_file = 'matches_kuri.csv' # Nome do CSV com as partidas do Titan
    combined_matches_file = 'kuri_lhcarva_matches.csv' # Nome do arquivo de saída combinado

    print("\n--- Combinando arquivos de PARTIDAS ---")
    if combine_csv_files(my_matches_file, titan_matches_file, combined_matches_file):
        print("Combinação de partidas concluída.")
    else:
        print("Falha na combinação de partidas.")

    # Exemplo 2: Combinar arquivos de participantes (participants.csv)
    # ATENÇÃO: Renomeie seus arquivos originais para evitar sobrescrever!
    my_participants_file = 'participants.csv'
    titan_participants_file = 'participants_kuri.csv'
    combined_participants_file = 'kuri_lhcarva_participants.csv'

    print("\n--- Combinando arquivos de PARTICIPANTES ---")
    if combine_csv_files(my_participants_file, titan_participants_file, combined_participants_file):
        print("Combinação de participantes concluída.")
    else:
        print("Falha na combinação de participantes.")

    print("\nLembre-se de verificar os arquivos combinados e carregá-los no Power BI!")